# API Nakala

## Packages

In [20]:
using CSV
using DataFrames
using ImageSegmentation
using HTTP
using JSON
using Dates
using FileIO

## Identifiants

In [13]:
path = @__DIR__
credentials = CSV.read(joinpath(path, "credentials.csv"), DataFrame, header=1) #liste des utilisateurs
user = "jmorvan" #choix de l'utilisateur (api test = tnakala)
usrCredentials = filter(:user => n -> n == user, credentials) #récupération des identifiants
apiKey = usrCredentials[1, :apikey] #clé API

"295d7fc9-8505-a85c-aa7f-4a490b95e758"

## API
Pour travailler avec l'API test, modifiez la valeur de la variable `apitest` par `true`

In [14]:
apitest = false

if apitest == true
  apiurl = "https://apitest.nakala.fr"
else 
  apiurl = "https://api.nakala.fr"
end

"https://api.nakala.fr"

## Chercher un fichier précis dans une donnée

In [15]:
title = "Z1J434"
lang = ""
firstImage = 1
lastImage = 2

scope = ["deposited", "owned", "shared", "editable", "readable", "all"]
#deposited : les données déposées par l'utilisateur (ROLE_DEPOSITOR)
#owned : les données dont l'utilisateur est propriétaire (ROLE_OWNER)
#shared : les données partagées avec l'utilisateur (ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER, mais pas ROLE_OWNER)
#editable : les données modifiables par l'utilisateur (ROLE_OWNER, ROLE_ADMIN ou ROLE_EDITOR)
#readable : les données lisibles par l'utilisateur (ROLE_OWNER, ROLE_ADMIN, ROLE_EDITOR ou ROLE_READER)
#all : toute
s = scope[1]


url = joinpath(apiurl, "users", "datas", s)

headers = Dict(
  "X-API-KEY" => apiKey,
  "Content-Type" => "application/json"
)

body = Dict(
  :page => 1,
  :limit => 100,
  :titleSearch => title,
  :titleSearchLang => lang
)

userDatas = HTTP.request("POST", url, headers, JSON.json(body))
userDatasResponse = JSON.parse(String(HTTP.payload(userDatas))) # réponse du server

datas = get(userDatasResponse, "data", "nothing")
# il peut y avoir plusieurs résultats
data = datas[1]
identifier = get(datas[1], "identifier", "")
files = get(data, "files", "")

991-element Vector{Any}:
 Dict{String, Any}("name" => "Z1J434_0001.JPG", "embargoed" => "2023-02-28T00:00:00+01:00", "humanReadableEmbargoedDelay" => Any[], "size" => 6474755, "mime_type" => "image/jpeg", "sha1" => "8941d1bc63ce6d9d03a93c9fc02909773ed9e685", "extension" => "jpg", "description" => nothing)
 Dict{String, Any}("name" => "Z1J434_0002.JPG", "embargoed" => "2023-02-28T00:00:00+01:00", "humanReadableEmbargoedDelay" => Any[], "size" => 6927522, "mime_type" => "image/jpeg", "sha1" => "a88fb1df1f1420ff538b6d1b2d46b8d3147ea519", "extension" => "jpg", "description" => nothing)
 Dict{String, Any}("name" => "Z1J434_0003.JPG", "embargoed" => "2023-02-28T00:00:00+01:00", "humanReadableEmbargoedDelay" => Any[], "size" => 7408610, "mime_type" => "image/jpeg", "sha1" => "cd137d9a0114a0980ba83db58cc1120fd7b1cd7e", "extension" => "jpg", "description" => nothing)
 Dict{String, Any}("name" => "Z1J434_0004.JPG", "embargoed" => "2023-02-28T00:00:00+01:00", "humanReadableEmbargoedDelay" => Any[

## Création d'un array pour les images

In [16]:
urls = Vector()
for i in firstImage:lastImage
  file = files[i]
  fileName = get(file, "name", "unknown")

  fileIdentifier = get(file, "sha1", "unknown")
  fileUrl = joinpath(apiurl, "data", identifier, fileIdentifier)

  img = Dict(
    "url" => fileUrl,
    "apiurl" => apiurl,
    "dataIdentifier" => identifier,
    "fileIdentifier" => fileIdentifier,
    "label" => fileName
  )

  push!(urls, img)
end

urls

2-element Vector{Any}:
 Dict("apiurl" => "https://api.nakala.fr", "label" => "Z1J434_0001.JPG", "dataIdentifier" => "10.34847/nkl.40c2ysl9", "fileIdentifier" => "8941d1bc63ce6d9d03a93c9fc02909773ed9e685", "url" => "https://api.nakala.fr/data/10.34847/nkl.40c2ysl9/8941d1bc63ce6d9d03a93c9fc02909773ed9e685")
 Dict("apiurl" => "https://api.nakala.fr", "label" => "Z1J434_0002.JPG", "dataIdentifier" => "10.34847/nkl.40c2ysl9", "fileIdentifier" => "a88fb1df1f1420ff538b6d1b2d46b8d3147ea519", "url" => "https://api.nakala.fr/data/10.34847/nkl.40c2ysl9/a88fb1df1f1420ff538b6d1b2d46b8d3147ea519")

## Fonction Split()

In [ ]:
function split(label, file, s)
  middle = s[2]/2
  l = floor(Int, middle+100)
  r = floor(Int, middle-100)
  leftPart = @view file[ :, 1:l]
  rightPart = @view file[ :, r:s[2]]
  save(string(splitext(label)[1], "-l", splitext(label)[2]), leftPart)
  save(string(splitext(label)[1], "-r", splitext(label)[2]), rightPart)
end

In [ ]:
for img in urls
  label = get(img, "label", "unknown")
  file = download(get(img, "url", "")) |> load
  s = size(file)
  
  file = rot180(file)

  if s[1] < s[2]
    split(label, file, s)
  else
    save(label, file)
  end

end

## Spliiit


In [21]:
function spliiit(img, data)
    
    segments = felzenszwalb(img, 5000, 300000) # le deuxième argument permet de supprimer les segments les plus petits
    
    segMap = labels_map(segments)
    
    pixel = sort(segment_pixel_count(segments); byvalue=true)
    v = collect(values(pixel))
    k = collect(keys(pixel))
    l = length(collect(values(pixel)))
    
    vec = Vector()
    for i in l-1:l
        coordinates = findall(x -> x == k[i], segMap)

        ymin, ymax = extrema(c -> c[1], coordinates)
        h = ymax - ymin

        xmin, xmax = extrema(c -> c[2], coordinates)
        w = xmax - xmin
        
        region = joinpath(get(data, "apiurl", "unknown"), "iiif", get(data, "dataIdentifier", "unknown"), get(data, "fileIdentifier", "unknown"), "$xmin,$ymin,$w,$h/full/0/default.jpg")
        push!(vec, region)

    end
    

    #draw!(file, Cross(Point(xmin, ymin), 300), RGB{N0f8}(1,0,1))
    #draw!(file, Cross(Point(xmax, ymax), 300), RGB{N0f8}(1,0,1))

    
    
    vec
end

#results = spliiit(img)

spliiit (generic function with 1 method)

In [23]:
for img in urls
  file = download(get(img, "url", "")) |> load
  s = size(file)

  if s[1] < s[2]
    spliiit(file, data)
  else
    println("todo")
  end

end